In [ ]:
import cv2 as cv
import numpy as np
import os

#img = cv.imread('images/20250204_153513.jpg')
#points = cv.findChessboardCorners(img,(9,6))
imgpoints = [] # inner corner coordinates
imgpoints_not_automatic = [] # inner corner coordinates for manual annotation 

cell_size = 24 # 2.4cm / 2.4mm # real word width/length of a square
objpoints = [] # 3D world coordinates
objp = np.zeros((9*6,3), np.float32) # Turn chess board into 3d world coordinate system
objp[:,:2] = np.mgrid[0:9,0:6].T.reshape(-1,2) 

objp = objp * cell_size # Multiply it by the real world width to get accurate coordinates

def visualize_estimations(estimated_points,img):
    # Visuzalize the interpolated points
    temp_img = img.copy()
    for point in estimated_points:
        x = point[0][0]
        y = point[0][1]
        cv.circle(temp_img,(int(x),int(y)), radius=10, color=(255,0,0),thickness=-1)

    cv.imshow('image',temp_img)
    cv.setMouseCallback('image', click_event) 
    cv.waitKey(0)

    cv.destroyAllWindows()

def interpolation(corners):
    ### Interpolate the points linearly
    TL,TR,BR,BL = corners # The corners coordinates
    
    ### calculate the pixel difference between squares for top and bottom
    top_offset_x = (TR[0] - TL[0]) / 10 
    top_offset_y = (TR[1] - TL[1]) / 10
    bott_offset_x = (BR[0] - BL[0]) / 10
    bott_offset_y = (BR[1] - BL[1]) / 10
    top_row = []
    bot_row = []
    
    ### Calculate all points in top / bottom row
    for i in range(11):
        x_top = TL[0] + i * top_offset_x 
        y_top = TL[1] + i * top_offset_y
        top_row.append((x_top,y_top))
        x_bott = BL[0] + i * bott_offset_x
        y_bott = BL[1] + i * bott_offset_y
        bot_row.append((x_bott,y_bott))

    interpolated_points = []
    ### Linearly interpolate between bottom and top points
    for i in range(6):
        for j in range(9):
            x = top_row[1:10][j][0] + ((bot_row[1:10][j][0] - top_row[1:10][j][0]) / 7) * (i+1)
            y = top_row[1:10][j][1] + ((bot_row[1:10][j][1] - top_row[1:10][j][1]) / 7) * (i+1)
            interpolated_points.append(np.array([[x,y]]))

    return np.array(interpolated_points, dtype='float32')


def click_event(event, x, y, flags, params):
    ### Function to manually annotate points
    if event == cv.EVENT_LBUTTONDOWN:
        font = cv.FONT_HERSHEY_SIMPLEX
        cv.circle(img, (x,y), radius=10, color=(255,0,0),thickness=-1)
        cv.putText(img, f'{x},{y}',(x,y), font, 1,(255, 255, 0), 2)
        cv.imshow('image', img)
        corners.append((x,y))
        if len(corners) == 4:
            cv.destroyAllWindows()

In [ ]:
### Find the corners of each image automatically or manually if needed
for i in os.listdir('images'):
    if i == '.ipynb_checkpoints':
        continue
    img = cv.imread('images/'+i)  
    img = cv.resize(img,(1280,720)) # Original image is large, so resize for annotation
    img = cv.cvtColor(img, cv.COLOR_BGR2GRAY) # GrayScale the image
    points = cv.findChessboardCorners(img,(9,6)) # Automatically find chessboard corners
    if points[0] == False: # If not found automatically
        corners = []
        print('no corners found')  
        cv.putText(img, 'No corners found, manually annotate corners (clockwise)', (0,100), cv.FONT_HERSHEY_SIMPLEX, 1,(0, 0, 255), 2)
        cv.imshow('image',img)
        cv.setMouseCallback('image', click_event) 
        cv.waitKey(0)
        cv.destroyAllWindows()
        #Interpolate manual points
        interpolated_points = interpolation(corners)
        imgpoints_not_automatic.append(interpolated_points)
        objpoints.append(objp)
        visualize_estimations(interpolated_points,img)
        
    else:
        imgpoints.append(points[1])
        objpoints.append(objp) 

In [ ]:
# Create subsets of images

# Run 1 
_, intr_para_1, distortion_1, rvec_1, tvec_1 = cv.calibrateCamera(objpoints, imgpoints+imgpoints_not_automatic, img.shape[1::-1], None, None)

# Run 2
_, intr_para_2, distortion_2, rvec_2, tvec_2 = cv.calibrateCamera(objpoints[:10], imgpoints[:10], img.shape[1::-1], None, None)


# Run 3
_, intr_para_3, distortion_3, rvec_3, tvec_3 = cv.calibrateCamera(objpoints[:5], imgpoints[:5], img.shape[1::-1], None, None)


In [ ]:
display(intr_para_1)
display(intr_para_2)
display(intr_para_3)

In [ ]:
# Using the reprojection error from opencv
# https://docs.opencv.org/4.x/dc/dbb/tutorial_py_calibration.html
def reprojection_error(rvecs,tvecs,intr_para,distortion,imgpoints,objpoints):
    mean_error = 0
    errors =[]
    for i in range(len(objpoints)):
        imgpoints2, _ = cv.projectPoints(objpoints[i], rvecs[i], tvecs[i], intr_para, distortion)
        error = cv.norm(imgpoints[i], imgpoints2, cv.NORM_L2) / len(imgpoints2)
        errors.append(error)
        mean_error += error
        print(error)
    print( "total error: {}".format(mean_error/len(objpoints)) )
    return errors
    
reprojection_error(rvec_1,tvec_1,intr_para_1,distortion_1,imgpoints+imgpoints_not_automatic, objpoints)
reprojection_error(rvec_2,tvec_2,intr_para_2,distortion_2,imgpoints[:10], objpoints[:10])
reprojection_error(rvec_3,tvec_3,intr_para_3,distortion_3,imgpoints[:5], objpoints[:5])

In [ ]:
i=0
for para, distortion in [(intr_para_1,distortion_1),(intr_para_2,distortion_2),(intr_para_3,distortion_3)]:
    ### Image loading and extrinsic calculation
    test_img = cv.imread('testimage.jpg') # Load test image
    test_img = cv.resize(test_img,(1280,720)) # Resize test image
    _, test_image_points = cv.findChessboardCorners(test_img,(9,6)) # Automatic corner detection
    _, rvec, tvec = cv.solvePnP(objp,test_image_points, para,distortion) # Caclulate camera extrinsics
    image = cv.drawFrameAxes(test_img, para, distortion, rvec, tvec, cell_size*5) # Draw the axes on the world origin
        
    ### Cube creation and drawing
    cube = np.float32([      # Values are multiplied by the size of a cell (to get the correct number of squares)
        [0, 0, 0],           # World origin
        [cell_size*2, 0, 0],   # X = 2, Y = 0, Z = 0
        [cell_size*2, cell_size*2, 0],  # X = 2, Y = 2, Z = 0
        [0, cell_size*2, 0],    # X = 0, Y = 2, Z = 0
        [0, 0, -cell_size*2],  # X = 0, Y = 0, Z = -2 (negate because the z axis is pointing inwards)
        [cell_size*2, 0, -cell_size*2],  # X = 2, Y = 0, Z = -2 
        [cell_size*2, cell_size*2, -cell_size*2],   # X = 2, Y = 2, Z = -2 
        [0, cell_size*2, -cell_size*2]     # X = 0, Y = 2, Z = -2 
    ])

    points, _ = cv.projectPoints(cube, rvec, tvec, para, distortion) # Project the points from 3d to 2d
    points = points.astype(int)

    edges = [(0,1), (1,2), (2,3), (3,0), (4,5), (5,6), (6,7), (7,4), (0,4), (1,5), (2,6), (3,7)] # Create all cube edges
    for edge in edges: # Draw a line for each edge
        cv.line(test_img, tuple(points[edge[0]][0]), tuple(points[edge[1]][0]), (255,255,0), 4)
        
    
    ### Draw the top plane
    R, _ = cv.Rodrigues(rvec) # Create rotation matrix
    plane_camera_coordinates = np.matmul(R, np.mean(cube[4:],axis=0)) + tvec.reshape(3) # center top plane to camera coordinates
    V_value = (4000 - np.linalg.norm(plane_camera_coordinates)) / 4000 * 255 # calculate euclidean distance and normalize
    S_value = (45 - np.degrees(np.arccos(np.dot(R[:,2], np.array([0,0,1]))))) / 45 * 255  # calculate the angle between the camera's perpendicular view and the plane
    H_value = np.arctan2(plane_camera_coordinates[0],plane_camera_coordinates[1]) # calculate the azimuth angle
    hsv_bgr = cv.cvtColor(np.uint8([[[H_value, S_value, V_value]]]), cv.COLOR_HSV2BGR) # turn HSV to bgr
    cv.fillConvexPoly(test_img,points[4:],color=(int(hsv_bgr[0][0][0]),int(hsv_bgr[0][0][1]),int(hsv_bgr[0][0][2]))) #Use the last half (upper part) of the cube    
    
    ### Show the images with drawn axes
    cv.imshow('image',test_img)
    cv.setMouseCallback('image', click_event) 
    cv.waitKey(0)
    cv.destroyAllWindows()
    
    ### Save image
    cv.imwrite(f'testimg{i}.jpg',test_img)
    i+=1

### CHOICE 6: produce a 3D plot with the locations of the camera relative to the chessboard 


In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
def visualize_positions(rvecs, tvecs):
    ### Visualize all camera positions and the chessboard origin
    fig = plt.figure() # Create figure
    ax = fig.add_subplot(projection='3d') 
    
    ### For each taken picture (rvec/tvec pair) translate the camera coordinates to world coordinates
    camera_positions = []
    for rvec, tvec in zip(rvecs, tvecs):
        R, _ = cv.Rodrigues(rvec)
        C = np.matmul(-R.T,tvec)
        camera_positions.append(C)
    camera_positions = np.array(camera_positions)
    
    ### Scatter all points with camera points blue and chessboard origin red
    ax.scatter(camera_positions.T[0][0], 
              camera_positions.T[0][1], 
              camera_positions.T[0][2], color = 'blue', label='Camera')
    ax.scatter(0,0,0,color = 'Red', label ='Chessboard origin')
    ax.legend()
    
cm = visualize_positions(rvec_1,tvec_1)